# Reading satellite raster files with Rasterio


In [ ]:
# import the required module
import os
import sys
import rasterio
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..')) # set default directory where this notebook is running from

Rasterio is a very useful module for raster processing for reading and writing several different raster formats in Python. 

Rasterio is based on GDAL and Python automatically registers all GDAL drivers. 
Commonly used file formats include e.g., TIFF and GeoTIFF, ASCII Grid and Erdas Imagine .img -files.

Landsat 8 bands are stored as separate GeoTIFF -files in the original package. Each band contains information of surface reflectance from different ranges of the electromagnetic spectrum.


installation of GDAL at https://anaconda.org/conda-forge/gdal

In [ ]:
#read raster file
fp= "Data/CPH_masked.tif"

# Open the file:
raster = rasterio.open(fp)

# Check type of the variable 'raster'
print ("The object type of the raster image is: ", type(raster))

You can see that our raster variable is a rasterio._io.RasterReader type, which means that we have opened the raster file ready for reading.


# Reading raster file properties


Let's explore the metadata of the image

In [ ]:
# Projection
raster.crs

check what projection it contains
https://epsg.io/?q=32632 

In [ ]:
# Dimensions --> width
raster.width

In [ ]:
# Dimensions --> height
raster.height

In [ ]:
# Number of bands
raster.count

In [ ]:
# Bounds of the file
raster.bounds

In [ ]:
# Driver (data format)
raster.driver

In [ ]:
# No data values for all channels
raster.nodatavals

In [ ]:
# All Metadata for the whole raster dataset
raster.meta

In [ ]:
type(raster.meta)

In [ ]:
# write a code to print each part of metadata in a single line
meta_dic= raster.meta

for k, v in meta_dic.items():
    print ("The ", k, "is ", v)

# Get raster bands


Different bands of a satellite images are often stacked together in one raster dataset. In our case, all 11 bands of the Landsat 8 scene are included in our GeoTIFF and the count is hence 11.
Remember landsat 8 bands (https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites?qt-news_science_products=0#qt-news_science_products)

In [ ]:
#Read the raster band as separate variable
band2 = raster.read(2)
band3 = raster.read(3)

In [ ]:
# Check type of the variable 'band'
type(band2)

In [ ]:
# Data type of the values
band2.dtype

have a look at raster data types at https://scikit-image.org/docs/dev/user_guide/data_types.html

In [ ]:
band2

# Band statistics


NOW, let’s check the values that are stored in the band. 
The values of the bands are stored as numpy arrays, so it is extremely easy to calculate basic statistics by using basic numpy functions.



In [ ]:
# Read all bands
array = raster.read()

In [ ]:
# Calculate statistics for each band
stats = []
for band in array:
    stats.append({
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})

In [ ]:
print(stats)

# Visualizing raster layers


Of course, it is always highly useful to take a look how the data looks like. This is easy with the "rasterio.plot.show()" function of rasterio.

In [ ]:
#import rasterio
from rasterio.plot import show
plt.figure(figsize=(20, 10))

fp= "Data/CPH_masked.tif"

# Open the file:
raster = rasterio.open(fp)

plt.figure(figsize=(20, 10))

show((raster, 3))

The image looks dark and we can not see clear content, so we should do image enhancement in order to strecth the diginal numbers (DN) mkaing a good contrast between grey values

In [ ]:
# Read the grid values into numpy arrays
red = raster.read(4)
green = raster.read(3)
blue = raster.read(2)

# Function to normalize the grid values
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

# Normalize the bands
redn = normalize(red)
greenn = normalize(green)
bluen = normalize(blue)

print("Normalized bands")
print(redn.min(), '-', redn.max(), 'mean:', redn.mean())
print(greenn.min(), '-', greenn.max(), 'mean:', greenn.mean())
print(bluen.min(), '-', bluen.max(), 'mean:', bluen.mean())

as you can see, the grey values are very close to the first quantile of values

In [ ]:
# Create RGB natural color composite
rgb = np.dstack((redn, greenn, bluen))

# Let's see how our color composite looks like
plt.figure(figsize=(20, 10))
plt.imshow(rgb)

In [ ]:
# Read the grid values into numpy arrays
swir2 = raster.read(7)
swir = raster.read(6)
nir = raster.read(5)
red = raster.read(4)
green = raster.read(3)

# Normalize the values using the function that we defined earlier
nirn = normalize(nir)
redn = normalize(red)
greenn = normalize(green)

# Create the composite by stacking
nrg = np.dstack((nirn, redn, greenn))

# Let's see how our color composite looks like
plt.figure(figsize=(20, 10))
plt.imshow(nrg)

### GDAL

GDAL is a translator library for raster and vector geospatial data formats that is released under an MIT style Open Source License by the Open Source Geospatial Foundation. As a library, it presents a single raster abstract data model and single vector abstract data model to the calling application for all supported formats. It also comes with a variety of useful command line utilities for data translation and processing.

[MORE](https://gdal.org)

In [ ]:
#Let's repeat some of these steps with GDAL
from osgeo import gdal
image_data = gdal.Open("Data/CPH_masked.tif")


total_number_of_bands = image_data.RasterCount
print ("total number of bands in this image is %s " %total_number_of_bands)
rows = image_data.RasterYSize
columns = image_data.RasterXSize
print ("rows: %s pixels" %columns)
print ("columns: %s pixels"  %rows)

In [ ]:
projection = image_data.GetProjection()
print (projection)

In [ ]:
#let's read some of the bands
band_1 = image_data.GetRasterBand(1).ReadAsArray()
band_2 = image_data.GetRasterBand(2).ReadAsArray()
band_3 = image_data.GetRasterBand(3).ReadAsArray()
band_4 = image_data.GetRasterBand(4).ReadAsArray()
band_5 = image_data.GetRasterBand(5).ReadAsArray()
band_6 = image_data.GetRasterBand(6).ReadAsArray()

In [ ]:
true_color = np.dstack([band_3, band_2, band_1])
color_infrared = np.dstack([band_4, band_3, band_2])
false_natural_color = np.dstack([band_5, band_4, band_3])
false_color_II = np.dstack([band_6, band_5, band_3])

In [ ]:
show((raster, 1), cmap='Reds', title ='landsat image')
show((raster, 2), cmap='Greens')
show((raster, 3), cmap='Blues')

# Histogram of the raster data


It is recommended to always look at the histogram of your data. It is as easy as using "rasterio.plot.show_hist()" function.

In [ ]:
from rasterio.plot import show_hist
plt.figure(figsize=(20, 10))


show_hist(raster, bins=50, lw=0.0, stacked=False, alpha=0.3,
          histtype='stepfilled' ,title="Histogram")

# Clipping or Masking a raster


One common task in raster processing is clipping raster files based on a Polygon. We are going to clip the raster data from a polygon.

In [ ]:
#import required packages
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs

We need to specify input and output filepaths


In [ ]:
# set the output path or filename
out_tif = r"CPH_masked_CLIP.tif"

WHY using "r" in the line above? (Since some of you faced this problem)

In Python, backslash is used to signify special characters.

e.g. "hello\nworld" -- the "\n" means a newline. Remember we used it for printing in a new line.

Path names on Windows tend to have backslashes in them. But we want them to mean actual backslashes, not special characters.

r stands for "raw" and will cause backslashes in the string to be interpreted as actual backslashes rather than special characters.

e.g. r"hello\nworld" literally means the characters "hello\nworld". Again, try printing it.

More info is in the Python docs, it's a good idea to search them for questions like these.

https://docs.python.org/3/tutorial/introduction.html#strings

Takeaway: ALWAYS put an "r" before a path/directory

Open the raster in read mode

In [ ]:
data = rasterio.open(fp)

In [ ]:
#Plot the data
plt.figure(figsize=(20, 10))

show((data, 2), cmap='terrain')

NOW, let's create a bounding box with Shapely for clipping


In [ ]:
# UTM coordinates EPSG: 32632
# http://bboxfinder.com/
from shapely.geometry import box

bbox = box(717621.8079, 6172470.4282, 731060.0044, 6180333.8423)


In [ ]:
import geopandas as gpd
# Insert the bbox into a GeoDataFrame
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs="EPSG:32632")

Next we need to get the coordinates of the geometry in such a format that rasterio wants them. This can be conducted easily with following function

In [ ]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

Get the geometry coordinates by using the function.


In [ ]:
coords = getFeatures(geo)

print(coords)

Ok, so rasterio wants to have the coordinates of the Polygon in this kind of format as above!



Now we are ready to clip the raster with the polygon using the coords variable that we just created. Clipping the raster can be done easily with the mask function that we imported in the beginning from rasterio, and specifying clip=True.

In [ ]:
# setting the output file and its ttransformation 
out_img , out_transform= mask(dataset=data, shapes=coords, crop=True)

Next, we need to modify the metadata. Let’s start by copying the metadata from the original data file.


In [ ]:
# Copy the metadata
out_meta = data.meta.copy()

print(out_meta)

Next we need to parse the EPSG value from the CRS so that we can create a Proj4 string using PyCRS library (to ensure that the projection information is saved correctly).


In [ ]:
data.crs.data

In [ ]:
# Parse EPSG code
epsg_code = int(data.crs.data['init'][5:])
print(epsg_code)

Now we need to update the metadata with new dimensions, transform (affine) and CRS (as Proj4 text)


In [ ]:
out_img.shape

In [ ]:
out_meta.update({"driver": "GTiff",
                 "height": out_img.shape[1],
                 "width": out_img.shape[2],
                 "transform": out_transform,
                 "crs": epsg_code})

Finally, we can save the clipped raster to disk with following command.


In [ ]:
with rasterio.open(out_tif, "w", **out_meta) as dest:
    dest.write(out_img)

Let’s still check that the result is correct by plotting our new clipped raster.


In [ ]:
clipped = rasterio.open(out_tif)

In [ ]:
plt.figure(figsize=(20, 10))
show((clipped, 5), cmap='terrain')


In [ ]:
clipped.meta

# Raster calculations


Calculations between bands of a raster file is a common GIS task. 
We will calculate NDVI (Normalized difference vegetation index) of Copenhagen based on our Landsat dataset. Conducting calculations with rasterio is fairly straightforward if the extent etc. matches because the values of the rasters are stored as numpy arrays (similar to the columns stored in Geo/Pandas, i.e. Series).


In [ ]:
# Let’s start by importing the necessary modules rasterio and numpy
from rasterio.plot import show

In [ ]:
# Let’s read the masked file.
fp = r"CPH_masked_CLIP.tif"

raster = rasterio.open(fp)

For calculating the NDVI (Normalized difference vegetation index) you need two bands:::
band-4 which is the Red channel and band-5 which is the Near Infrared (NIR)

In [ ]:
#Let’s read those bands from our raster source
red = raster.read(4)
nir = raster.read(5)

In [ ]:
red

In [ ]:
nir

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

plt.figure(figsize=(20, 10))

imgplot = plt.imshow(nir, norm=LogNorm(),  cmap='summer')
plt.colorbar()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

plt.figure(figsize=(20, 10))

imgplot = plt.imshow(red, norm=LogNorm(), cmap='summer')
plt.colorbar()

In [ ]:
# Now we are ready to calculate the NDVI. 
# First, we can create a filter where we calculate the values on such pixels that have a value larger than 0.

check = np.logical_or ( red > 0, nir > 0 )


In [ ]:
#Now we can apply that filter and calculate the ndvi index.
ndvi = np.where ( check,  (nir - red ) / ( nir + red ), -999 )

In [ ]:
ndvi

In [ ]:
#let's calcuate mean of ndvi
ndvi.mean()

In [ ]:
#let's calcuate SD of ndvi
ndvi.std()

you can look for colormaps at https://matplotlib.org/3.1.1/tutorials/colors/colormaps.html

In [ ]:
ndvi_pure = np.logical_and ( -1 <= ndvi, ndvi <= 1)

In [ ]:
plt.figure(figsize=(20, 10))

imgplot = plt.imshow(ndvi, norm=LogNorm(), cmap='PiYG')
plt.colorbar()

## Density slicing

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.pyplot as plt


# Create a colormap from a list of colors
colors = ['white','blue','grey','green']

cmap = ListedColormap(colors)

class_bins = [-1, -0.5, 0.02, 0.3, 0.8]
norm = BoundaryNorm(class_bins,len(colors))

plt.figure(figsize=(20, 10))

imgplot = plt.imshow(ndvi, norm=norm,  cmap=cmap)
plt.colorbar()


## Linear stretch

an alternative way of linear streching using Scikit-image library 

In [ ]:
plt.figure()
plt.figure(figsize=(20, 10))

plt.imshow(rgb)

In [ ]:
from skimage import exposure
import matplotlib.pyplot as plt
import numpy as np

rgb = np.dstack((redn, greenn, bluen))

def linearStretch(input, percent):
    pLow, pHigh = np.percentile(input[~np.isnan(input)], (percent, 100 - percent))
    img_rescale = exposure.rescale_intensity(input, in_range=(pLow, pHigh))
    return img_rescale

img_rescaled = linearStretch(rgb, 1.15)
plt.figure(figsize=(20, 10))
plt.imshow(img_rescaled)


Here is a histogram equalization solution that is working better than linear stretch.



In [ ]:
# Some of my images have NaNs for NoData so I have to remove them when creating the histogram
shape = rgb.shape
rgb_vector = rgb.reshape([rgb.shape[0] * rgb.shape[1], rgb.shape[2]])
rgb_vector = rgb_vector[~np.isnan(rgb_vector).any(axis=1)]

# View histogram of RGB values
fig = plt.figure(figsize=(10, 7))
fig.set_facecolor('white')
for color, channel in zip('rgb_vector', np.rollaxis(rgb, axis=-1)):
    counts, centers = exposure.histogram(channel)
    plt.plot(centers[1::], counts[1::], color=color)
plt.show()

# Get cutoff values based on standard deviations. Ideally these would be on either side of each histogram peak and cutoff the tail. 
lims = []
for i in range(3):
    x = np.mean(rgb_vector[:, i])
    sd = np.std(rgb_vector[:, i])
    low = x-(1.75*sd)  # Adjust the coefficient here if the image doesn't look right
    high = x + (1.75 * sd)  # Adjust the coefficient here if the image doesn't look right
    if low < 0:
        low = 0
    if high > 1:
        high = 1
    lims.append((low, high))

r = exposure.rescale_intensity(rgb[:, :, 0], in_range=lims[0])
g = exposure.rescale_intensity(rgb[:, :, 1], in_range=lims[1])
b = exposure.rescale_intensity(rgb[:, :, 2], in_range=lims[2])
rgb_enhanced = np.dstack((r, g, b))
plt.figure(figsize=(20, 10))
plt.imshow(rgb_enhanced)